<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/learnelixir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ

elixir を齧る。かじる。

今のイメージ erlang 上で、erlang は 並行処理のためのシステムで、その erlang 上で理想的な言語を作ろうとしたら、ruby + clojure みたいな言語になった。

Dave Thomas と まつもとゆきひろ が勧めているのだからいい言語なのだろう。


* https://elixirschool.com/ja/lessons/basics/basics/
* https://elixirschool.com/ja/lessons/basics/control-structures/
* https://magazine.rubyist.net/articles/0054/0054-ElixirBook.html

In [ ]:
%%capture
!sudo apt install elixir

メモ

このあと `!elixir --version` とするとエラーになるが、以下のコードセルに見るようにとりあえず使えているので先に進む。

`!elixir -h` としたら ワンライナー `elixir -e` が使えるらしいことがわかった。

`iex` というのがインタラクティブ環境なのだが、colab では使いにくいのでとりあえず使わない。


In [ ]:
!elixir -e 'IO.puts 3 + 3'
!elixir -e 'IO.puts "hello world!"'

6
hello world!


In [ ]:
%%writefile temp.exs
IO.puts "this is a pen."

Writing temp.exs


In [ ]:
!elixir temp.exs

this is a pen.


ネットで紹介されていた次のコードセルのコードはどうやって実行するのだろう。

説明:

このプログラムでは、Parallel というモジュールに pmap という関数を定義しています。 pmap は、与えられたコレクションに対して map（Ruby での Enumerable#map と同じようなものと考えて下さい）を行なうのですが、 各要素の処理を、要素数の分だけプロセスを生成し、各プロセスで並行に実行する、というものです。 ちょっと見ても、よくわからないような気がしますが、大丈夫、本書を読めば、わかるようになります。

とのこと。

In [ ]:
%%writefile temp.exs
defmodule Parallel do
  def pmap(collection, func) do
    collection
    |> Enum.map(&(Task.async(fn -> func.(&1) end)))
    |> Enum.map(&Task.await/1)
  end
end

コメントは `#`

In [ ]:
%%writefile temp.exs
# コメント実験
str = "helloworld!!!!"
IO.puts str

Overwriting temp.exs


In [ ]:
!elixir temp.exs

helloworld!!!!


In [ ]:
!elixir -e 'IO.puts 0b1111'
!elixir -e 'IO.puts 0o7777'
!elixir -e 'IO.puts 0xffff'
!elixir -e 'IO.puts 1000_000_00_0'

15
4095
65535
1000000000


In [ ]:
!elixir -e 'IO.puts 1.532e-4'
# .0 とか 1. とかはエラーになる

1.532e-4
** (TokenMissingError) nofile:1: syntax error: expression is incomplete
    (elixir) lib/code.ex:168: Code.eval_string/3



In [ ]:
!elixir -e 'IO.puts "日本語が書けますか"'
!elixir -e 'IO.puts "日本語が書けます"'

日本語が書けますか
日本語が書けます


In [ ]:
# 関数の括弧の前にスペースはエラーになる
!elixir -e 'IO.puts(0b1111)'
!elixir -e 'IO.puts("にほんご\n日本語")'
!elixir -e "IO.puts('にほんご\n\"日本語\"')"

15
にほんご
日本語
にほんご
"日本語"


In [ ]:
# 文字連結
!elixir -e 'IO.puts("ABCD"<>"EFGH")'

ABCDEFGH


値の埋め込み

`#{変数名}` を記述することで、変数の値を埋め込むことができる。


In [ ]:
!elixir -e 'val = 1000; IO.puts "val = #{val}"'

val = 1000


真理値

true と false で false と nil が false でそれ以外は true



In [ ]:
!elixir -e 'if true do IO.puts "true" end'
!elixir -e 'if True do IO.puts "true" end'
!elixir -e 'if False do IO.puts "true" end'
!elixir -e 'if false do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if nil do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if 0 do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if (-1) do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if [] do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if "" do IO.puts "true" else IO.puts "false" end'

true
true
true
false
false
true
true
true
true


`null` はない。

# アトム

アトムは名前がそのまま値となる定数である。

名前の前にコロン `:` をつけることでアトムになる。

アトムはアンダースコア `_` もしくは英字で始まる。 そのあとは
数字、アンダースコア、`@` が使える。 終端文字としてのみ「!」や「?」が使える。


In [ ]:
!elixir -e 'IO.puts true === :true'

true


演算子

In [ ]:
!elixir -e 'IO.puts 1 + 2'
!elixir -e 'x = 10; IO.puts x + 1'
!elixir -e 'IO.puts 1 - 2'
!elixir -e 'x = 10; IO.puts x - 1'
!elixir -e 'IO.puts 5 * 2'
!elixir -e 'x = 10; IO.puts x * 4'
!echo 
!elixir -e 'IO.puts 5 / 2'
!elixir -e 'x = 10; IO.puts x / 3'

3
11
-1
9
10
40

2.5
3.3333333333333335


In [ ]:
# 浮動少数ではなく整数としての結果がほしい場合は div 関数を使用
!elixir -e 'IO.puts div(10,5)'
!elixir -e 'IO.puts div(10,4)'
# 割り算の余り、剰余を求める場合は rem関数を使用します。
!elixir -e 'IO.puts rem(10,4)'
!elixir -e 'IO.puts rem(10,3)'
!elixir -e 'IO.puts rem(10,2)'

2
2
2
1
0


In [ ]:
# 比較演算子
!elixir -e 'IO.puts 1 == 1'
!elixir -e 'IO.puts 1 != 1'
!elixir -e 'IO.puts ! (1 != 1)'
!elixir -e 'IO.puts 20.0 == 20'
!elixir -e 'IO.puts 20.0 === 20'
!elixir -e 'IO.puts 20.0 !== 20'

true
false
true
true
false

true


In [ ]:
# 論理演算子
# 論理和
!elixir -e 'IO.puts "ABC" == "ABC" || 20 == 30'
!elixir -e 'IO.puts "ABC" == "abc" || 20 == 30'
!echo
# 論理積
!elixir -e 'IO.puts "ABC" == "ABC" && 20 == 20'
!elixir -e 'IO.puts "ABC" == "ABC" && 20 == 30'
!elixir -e 'IO.puts "ABC" == "def" && 10 > 100'
!echo
# 否定
!elixir -e 'IO.puts !("ABC" == "ABC")'
!elixir -e 'IO.puts !("ABC" == "DEF")'

true
false

true
false
false

false
true


# リスト

elixir のリストは複数の型を含むことができる。

メモ 下のコードセルで `IO.inspect` を使っているが、`IO.puts` ではリストを出力できなかったので、とりあえずわからないまま使っている。

In [ ]:
# リスト
!elixir -e 'IO.inspect [3.14, :pie, "Apple"]'


[3.14, :pie, "Apple"]


In [ ]:
# リスト先頭への追加(高速)
!elixir -e 'IO.inspect ["π" | [3.14, :pie, "Apple"]]'
# リスト末尾への追加(低速)
!elixir -e 'IO.inspect [3.14, :pie, "Apple"] ++ ["Cherry"]'

["π", 3.14, :pie, "Apple"]
[3.14, :pie, "Apple", "Cherry"]


上と下のコードセルでリストの連結を行っているが、++/2 演算子を用いている。 この `++/2` という表記は `++` が演算子自体で `/2` がアリティ (引数の数) を表す。 後述。

In [ ]:
# リストの連結
!elixir -e 'IO.inspect  [1, 2] ++ [3, 4, 1]'

[1, 2, 3, 4, 1]


In [ ]:
# リストの減算 
#  --/2 演算子は存在しない値を引いてしまってもオッケー
!elixir -e 'IO.inspect ["foo", :bar, 42] -- [42, "bar"]'
# 重複した値の場合、右辺の要素のそれぞれに対し、左辺の要素のうち初めて登場した同じ値が順次削除
!elixir -e 'IO.inspect [1,2,2,3,2,3] -- [1,2,3,2]'
# リストの減算の値のマッチには strict comparison が使われている
!elixir -e 'IO.inspect  [2] -- [2.0]'
!elixir -e 'IO.inspect  [2.0] -- [2.0]'

["foo", :bar]
[2, 3]
[2]
[]


In [ ]:
# head /tail
!elixir -e 'IO.inspect hd [3.14, :pie, "Apple"]'
!elixir -e 'IO.inspect tl [3.14, :pie, "Apple"]'

3.14
[:pie, "Apple"]


リストを頭部と尾部に分けるのに

* パターンマッチング
* cons 演算子( `|` )

を使うこともできる。

In [ ]:
!elixir -e '[head | tail] = [3.14, :pie, "Apple"]; IO.inspect head; IO.inspect tail'

3.14
[:pie, "Apple"]


# タプル

タプルの要素はメモリ上に隣接して格納される。

このため、長さを得るのは高速ｄが、修正は高コストになる。
なぜなら、全ての要素がコピーされるから。

タプルは波括弧 brace を用いて定義される。


In [ ]:
!elixir -e 'IO.inspect {3.14, :pie, "Apple"}'

{3.14, :pie, "Apple"}


タプルは関数の返り値に便利に利用される。

パターンマッチングと組み合わせて使われる。

```
iex> File.read("path/to/existing/file")
{:ok, "... contents ..."}
iex> File.read("path/to/unknown/file")
{:error, :enoent}
```



# キーワードリスト

キーワードリストとマップは elixir の連想配列である。

キーワードリストは最初の要素がアトムのタプルからなる特別なリストで、リストと同様の性能になる。

In [26]:
# キーワードリスト
!elixir -e 'IO.inspect [foo: "bar", hello: "world"]'
!elixir -e 'IO.inspect [{:foo, "bar"}, {:hello, "world"}]'
!elixir -e 'IO.inspect [foo: "bar", hello: "world"] == [{:foo, "bar"}, {:hello, "world"}]'

[foo: "bar", hello: "world"]
[foo: "bar", hello: "world"]
true


キーワードリストの 3 つの特徴

* キーはアトムである。
* キーは順序付けされている。
* キーの一意性は保証されない。

こうした理由から、キーワードリストは関数にオプションを渡すためによく用いられる。


# いまここ

# マップ

* キーワードリストとは違ってどんな型のキーも使える。
* 順序付けさない。
* キーの一意性が保証されている。重複したキーが追加された場合は、前の値が置き換えられる。
* 変数をマップのキーにできる。
* `%{}` 構文で定義する。



In [30]:
!elixir -e 'IO.inspect %{:foo => "bar", "hello" => :world}'
!elixir -e 'map = %{:foo => "bar", "hello" => :world}; IO.inspect map[:foo]'
!elixir -e 'map = %{:foo => "bar", "hello" => :world}; IO.inspect map["hello"]'
!echo
!elixir -e 'key = "hello"; IO.inspect %{key => "world"}'
!elixir -e 'IO.inspect %{:foo => "bar", :foo => "hello world"}'

%{:foo => "bar", "hello" => :world}
"bar"
:world

%{"hello" => "world"}
%{foo: "hello world"}


In [31]:
!elixir -e 'IO.inspect %{foo: "bar", hello: "world"} == %{:foo => "bar", :hello => "world"}'

true


In [ ]:

Copy
iex> key = "hello"
"hello"
iex> %{key => "world"}
%{"hello" => "world"}
重複したキーが追加された場合は、前の値が置き換えられます:

Copy
iex> %{:foo => "bar", :foo => "hello world"}
%{foo: "hello world"}
上記の出力からわかるように、アトムのキーだけを含んだマップには特別な構文があります:

Copy
iex> %{foo: "bar", hello: "world"}
%{foo: "bar", hello: "world"}

iex> %{foo: "bar", hello: "world"} == %{:foo => "bar", :hello => "world"}
true
加えて、アトムのキーにアクセスするための特別な構文もあります:

Copy
iex> map = %{foo: "bar", hello: "world"}
%{foo: "bar", hello: "world"}
iex> map.hello
"world"
マップのもう一つの興味深い特性は、マップの更新のための
固有の構文があることです(注: 更新と言っていますが、新しいmapが作成されます):

Copy
iex> map = %{foo: "bar", hello: "world"}
%{foo: "bar", hello: "world"}
iex> %{map | foo: "baz"}
%{foo: "baz", hello: "world"}
注意: この構文は、マップに既に存在するキーを更新する場合にのみ機能します！キーが存在しない場合、 KeyError が発生します。

新しいキーを作成するには、代わりに Map.put/3 を使用します。

Copy
iex> map = %{hello: "world"}
%{hello: "world"}
iex> %{map | foo: "baz"}
** (KeyError) key :foo not found in: %{hello: "world"}
    (stdlib) :maps.update(:foo, "baz", %{hello: "world"})
    (stdlib) erl_eval.erl:259: anonymous fn/2 in :erl_eval.expr/5
    (stdlib) lists.erl:1263: :lists.foldl/3
iex> Map.put(map, :foo, "baz")
%{foo: "baz", hello: "world"}
Caught a mistake or want to contribute to the lesson? Edit this page on GitHub!
← 基本
Enum →
bg bn de en es fr gr id it ja ko ms no pl pt ru sk ta th tr uk vi zh-hans zh-hant
Elixir 1.10.1 - Erlang/OTP 22.0 [erts-10.5.3]
Menu
プログラミング言語Elixirのレッスン
© 2021 Sean Callan All rights reserved.

Toggle